# Полиномиальная регрессия (Polynomial Regression) и регуляризация

In [1]:
import numpy as np
from sklearn import linear_model
from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn import preprocessing
# возьмем данные о домах в Калифорнии
from sklearn.datasets import fetch_california_housing

## Полиномиальная регрессия (Polynomial Regression)

> **Полиномиальная регрессия (Polynomial Regression)** — это более сложная модель, чем линейная регрессия. \
Вместо уравнения прямой используется уравнение полинома (многочлена). \
Степень полинома может быть сколь угодно большой: чем больше степень, тем сложнее модель.

Например, если в исходных данных было два признака $x_1$ и $x_2$.\
То после генерации полиномиальных признаков мы получим 5 признаков:\
$ x_1, x_2, x_1^2, x_2^2, x_1x_2,$

Полиномиальные признаки можно создать с помощью объекта класса [PolynomialFeatures](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.PolynomialFeatures.html) 
из модуля `preprocessing`. \
Это преобразователь, который позволяет сгенерировать полиномиальные признаки любой степени \
и добавить их в таблицу. \
У него есть два важных параметра:

- `degree` — степень полинома. По умолчанию используется степень `2` \
(лучше не указывать больше `3`-х, так как возрастает риск переобучения).
- `include_bias` — включать ли в результирующую таблицу столбец из единиц (x в степени 0). \
По умолчанию стоит `True`, лучше выставить его в значение `False`, \
так как столбец из единиц и так добавляется в методе наименьших квадратов.

Получим данные о домах в калифорнии и подготовим их для передачи в модель.

In [2]:
data = fetch_california_housing(as_frame=True)

housing_data = data['frame']
housing_data.head()

,MedInc,HouseAge,AveRooms,AveBedrms,Population,AveOccup,Latitude,Longitude,MedHouseVal
0,8.3252,41.0,6.984127,1.023810,322.0,2.555556,37.88,-122.23,4.526
1,8.3014,21.0,6.238137,0.971880,2401.0,2.109842,37.86,-122.22,3.585
2,7.2574,52.0,8.288136,1.073446,496.0,2.802260,37.85,-122.24,3.521
3,5.6431,52.0,5.817352,1.073059,558.0,2.547945,37.85,-122.25,3.413
4,3.8462,52.0,6.281853,1.081081,565.0,2.181467,37.85,-122.25,3.422


In [3]:
# признаки
X = housing_data.drop(['MedHouseVal'], axis=1)
# целевой признак
y = housing_data['MedHouseVal']

# разделяем на тренировочную и тестовую выборки
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)

# нормализуем данные, чтобы коэффициенты не были слишком большими, так модель будет устойчивой

# инициализируем нормализатор MinMaxScaler
mm_scaler = preprocessing.MinMaxScaler()

# производим стандартизацию
mm_scaler.fit(X_train)

X_train_scaled = mm_scaler.transform(X_train)
X_test_scaled = mm_scaler.transform(X_test)

In [4]:
# cоздаём генератор полиномиальных признаков
polynomial_features_generator2 = preprocessing.PolynomialFeatures(degree=2, include_bias=False)
# обучаем его
polynomial_features_generator2.fit(X_train_scaled)

# генерируем полиномиальные признаки для тренировочной выборки
X_train_poly2 = polynomial_features_generator2.transform(X_train_scaled)

# генерируем полиномиальные признаки для тестовой выборки
X_test_poly2 = polynomial_features_generator2.transform(X_test_scaled)

# посмотрим на количество созданных признаков
print(X_train_poly2.shape[1])
print(X_test_poly2.shape[1])

44
44


Тип возвращаемых данных - `numpy`-массив.

In [5]:
print(type(X_train_poly2))
print(type(X_test_poly2))

<class 'numpy.ndarray'>
<class 'numpy.ndarray'>


Передадим новые признаки в линейную регрессию.

In [6]:
# создаём объект класса LinearRegression
linear_regression_poly2_model = linear_model.LinearRegression()

# обучаем модель
linear_regression_poly2_model.fit(X_train_poly2, y_train)

# делаем предсказание для тренировочной выборки
y_train_pred_poly2 = linear_regression_poly2_model.predict(X_train_poly2)

# делаем предсказание для тестовой выборки
y_test_pred_poly2 = linear_regression_poly2_model.predict(X_test_poly2)

# посчитаем MAPE для тренировочной выборки
mape_train2 = metrics.mean_absolute_percentage_error(y_train, y_train_pred_poly2)
# посчитаем MAPE для тренировочной выборки
mape_test2 = metrics.mean_absolute_percentage_error(y_test, y_test_pred_poly2)

print('MAPE train:', mape_train2)
print('MAPE test:', mape_test2)

MAPE train: 0.2616542118066179
MAPE test: 0.26992984576487733


Метрики примерно одинаковы, значит переобучения нет.\
Значение метрики лучше, чем для [обычной линейной регрессии](./linear_regression.ipynb) (0.31978371646123604)

In [7]:
np.round(linear_regression_poly2_model.coef_[1], decimals=5)

3.51143

In [8]:
# свободный член w0
print(f'w0: {np.round(linear_regression_poly2_model.intercept_, 5)}') 

# остальные параметры модели w1, w2, ..., wm
print(f'wi: {linear_regression_poly2_model.coef_}')

w0: 3.1787
wi: [ 2.52905745e+01  3.51142514e+00 -1.52130188e+02  1.28251194e+02
 -1.57696433e+01 -3.57188251e+02 -7.00091468e+00 -5.94615452e+00
 -6.43248386e+00  1.18175064e+00  7.91187994e+01 -5.08985805e+01
  2.64578650e+01 -6.92815054e+01 -2.20445437e+01 -2.20883158e+01
  5.48888465e-01 -4.30141227e+00  1.40765257e+01  4.66868674e+00
 -1.24963165e+02 -4.95963480e+00 -5.15429266e+00  2.32407810e+02
 -4.15478439e+02 -2.77596683e+02  3.74288846e+03  1.41818422e+02
  1.42437016e+02  1.84909186e+02  4.41396198e+02 -2.72119328e+03
 -1.28448345e+02 -1.26053300e+02  1.80790851e+00  1.08219429e+03
  3.52308423e+00  1.30415893e+00  8.12143282e+01  2.38042139e+02
  1.98097051e+02  5.12905889e+00  9.59439228e+00  3.84716685e+00]


Коэффициенты не очень большие, можно предположить, что модель устойчива.

## Регуляризация

> **Регуляризация** — способ уменьшения переобучения моделей машинного обучения.

Осуществляется путем намеренного увеличения смещения модели, чтобы уменьшить ее разброс.\
К функции потерь добавляется неотрицательное слагаемое (называемое также **штрафом**).\
Эта слагаемое специально повышает ошибку.\
В результате алгоритм находит не истинный минимум, а псевдоминимум.

Есть несколько способов добавления штрафа к функции потерь:

- **L1-регуляризация (Lasso)** — добавление к функции потерь суммы модулей коэффициентов, \
умноженных на коэффициент регуляризации $\alpha$:

$$ L_{1}(w)=M S E+\alpha \sum_{j=1}^{m}\left|w_{j}\right| $$

- **L2-регуляризация (Ridge)**, или регуляризация Тихонова — \
добавление к функции потерь суммы квадратов коэффициентов, \
умноженных на коэффициент регуляризации $\alpha$\:

$$ L_{2} (w) = MSE + \alpha \sum_{j=1}^{m} (w_{j})^{2} $$

- **Эластичная сетка (Elastic Net)** — это комбинация из двух методов регуляризации. \
Функция потерь в таком методе выглядит следующим образом:

$$ L_{2}(w)=M S E+\alpha \cdot \lambda \sum_{i=1}^{m}\left|w_{i}\right|+\alpha \cdot(1-\lambda) \sum_{i=1}^{m}\left(w_{i}\right)^{2} \rightarrow \min _{w} $$

Параметры $\alpha$ и $\lambda$ позволяют регулировать вклад L1- и L2-регуляризации.\
Способ используется редко, так как требует подбора сразу двух оптимальных значений параметров ($\alpha$ и $\lambda$).

> **Коэффициенты** ($\alpha$ и $\lambda$) — это коэффициенты регуляризации. \
Они отвечают за то, насколько сильное смещение вносится в модель: \
чем оно больше, тем сильнее будет штраф за переобучение.

Если параметра у модели только два ($w_0$ и $w_1$)\
то геометрически в случае L1-регуляризации минимум функции потерь ищется на пересечении с ромбом в центре координат,\
а в случае L2-регуляризации - с кругом.

Поиск минимума строится на основе [метода множителей Лагранжа](https://ru.wikipedia.org/wiki/%D0%9C%D0%B5%D1%82%D0%BE%D0%B4_%D0%BC%D0%BD%D0%BE%D0%B6%D0%B8%D1%82%D0%B5%D0%BB%D0%B5%D0%B9_%D0%9B%D0%B0%D0%B3%D1%80%D0%B0%D0%BD%D0%B6%D0%B0).\
В sklearn для решения задачи оптимизации 
используется итеративный алгоритм [координатного спуска](https://ru.wikipedia.org/wiki/%D0%9A%D0%BE%D0%BE%D1%80%D0%B4%D0%B8%D0%BD%D0%B0%D1%82%D0%BD%D1%8B%D0%B9_%D1%81%D0%BF%D1%83%D1%81%D0%BA).

В **sklearn** методы регуляризации реализованы в классах:
- [Lasso](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.Lasso.html) - L1-регуляризация
(параметр регуляризации `alpha` по умолчанию равен `1`, но лучше брать `< 1`, например `0.1`).
- [Ridge](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.Ridge.html) - L2-регуляризация
(параметр регуляризации `alpha` по умолчанию равен `1`, но лучше брать `< 1`, например `0.1`).
- [ElasticNet](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.ElasticNet.html) - Эластичная сетка (параметры по умолчанию `alpha=1.0`, `l1_ratio=0.5`)
- [SGDRegressor](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.SGDRegressor.html) - регуляризация присутствует по умолчанию.\
В инициализаторе класса есть параметр `penalty`, который принимает тип регуляризации (`l1`, `l2` и `elasticnet`). \
По умолчанию используется L2-регуляризация (`penalty='l2'`). \
Коэффициент регуляризации `alpha` по умолчанию равен `0.0001` (относительно слабая регуляризация).

Для примера создадим полиномиальные признаки третьей степени и решим проблему переобучения с помощью `Lasso`.

In [9]:
# cоздаём генератор полиномиальных признаков
polynomial_features_generator3 = preprocessing.PolynomialFeatures(degree=3, include_bias=False)
# обучаем его
polynomial_features_generator3.fit(X_train_scaled)

# генерируем полиномиальные признаки для тренировочной выборки
X_train_poly3 = polynomial_features_generator3.transform(X_train_scaled)

# генерируем полиномиальные признаки для тестовой выборки
X_test_poly3 = polynomial_features_generator3.transform(X_test_scaled)

# посмотрим на количество созданных признаков
print(X_train_poly3.shape[1])
print(X_test_poly3.shape[1])

164
164


In [10]:
# создаём объект класса LinearRegression
linear_regression_poly3_model = linear_model.LinearRegression()

# обучаем модель
linear_regression_poly3_model.fit(X_train_poly3, y_train)

# делаем предсказание для тренировочной выборки
y_train_pred_poly3 = linear_regression_poly3_model.predict(X_train_poly3)

# делаем предсказание для тестовой выборки
y_test_pred_poly3 = linear_regression_poly3_model.predict(X_test_poly3)

# посчитаем MAPE для тренировочной выборки
mape_train3 = metrics.mean_absolute_percentage_error(y_train, y_train_pred_poly3)
# посчитаем MAPE для тренировочной выборки
mape_test3 = metrics.mean_absolute_percentage_error(y_test, y_test_pred_poly3)

print('MAPE train:', mape_train3)
print('MAPE test:', mape_test3)

MAPE train: 0.24145398330869788
MAPE test: 0.31314691854350013


На тренировочной выборке модель в среднем ошибается на 24%\
А на тестовой на 31%\
Попробуем исправить это с помощью регуляризации.

In [11]:
# Создаём объект класса линейной регрессии с L1-регуляризацией
lasso_poly3_model = linear_model.Lasso(alpha=0.0001, max_iter=10000)

# Обучаем модель
lasso_poly3_model.fit(X_train_poly3, y_train)

# Делаем предсказание для тренировочной выборки
y_train_pred_poly3 = lasso_poly3_model.predict(X_train_poly3)

# Делаем предсказание для тестовой выборки
y_test_pred_poly3 = lasso_poly3_model.predict(X_test_poly3)

# посчитаем MAPE для тренировочной выборки
mape_train3 = metrics.mean_absolute_percentage_error(y_train, y_train_pred_poly3)
# посчитаем MAPE для тренировочной выборки
mape_test3 = metrics.mean_absolute_percentage_error(y_test, y_test_pred_poly3)

print('MAPE train:', mape_train3)
print('MAPE test:', mape_test3)

MAPE train: 0.2805644624591604
MAPE test: 0.2858950436023314


Метрики на тренировочной и тестовой выборках примерно равны.\
Мы решили проблему переобучения с помощью регуляризации.

In [13]:
# свободный член w0
print(f'w0: {np.round(lasso_poly3_model.intercept_, 5)}') 

# остальные параметры модели w1, w2, ..., wm
print(f'wi: {np.round(lasso_poly3_model.coef_, 5)}')

w0: 4.88571
wi: [  3.14523   0.5445  -13.48666  14.0862    0.       -2.99935  -6.45283
  -5.01515  16.18719   6.79688   0.        0.        0.14411  -0.
  -2.95885  -5.35546   0.       -0.        2.428     0.       -0.
  -0.11338  -0.9973    0.        0.       -0.       -0.       -0.
  -0.       -0.        0.       -0.       -0.        0.       -0.
  -0.        0.       -0.        0.       -0.       -0.       -0.
   0.        0.      -11.39623  -6.02127   0.        0.        0.
  -0.       -1.58879   0.        0.       -0.        0.        0.
  -0.       -2.80937  -0.        0.        0.        0.       -0.
   0.        0.       -0.        0.       -0.       -0.        0.
   0.       -0.        0.        0.       -0.       -0.       -0.
  -0.       -0.21634  -0.        1.7965   -0.        0.        0.67163
  -0.       -2.2962   -2.92284   0.        0.       -0.       -0.
  -0.       -0.        0.        0.       -0.        0.        0.
   0.        0.        0.       -0.       -0.     

Большая часть коэффициентов обнулилась. \
Это значит, что признаки, которые соответствуют этим коэффициентам, \
не используются в прогнозе модели Lasso-регрессии.\
Регуляризация Ridge не обнуляет коэффициенты, она использует все признаки.